# Scraping data from [OrganiCity](http://organicity.eu/)- [SmartSantander](http://www.smartsantander.eu/)


[Discovery API](https://discovery.organicity.eu/) can be used to find entities of different cities. Once, the entities are determined it is possible to get [historic data](http://data.organicity.eu/) through a more straightforward option.   

### Basic queries

| Description | Query   |
|------|------|
|Filter assets from a site by type | https://discovery.organicity.eu/v0/assets/sites/santander?type=< TYPE \> |
|Query attributes of an asset with historical values | https://data.organicity.eu/< ASSET_ID \>/attributes |
|Query historical values from attribute in an asset | https://data.organicity.eu/< ASSET_ID \>/< ATTRIBUTE \>/readings |

### SmartSantander types

| Type |Attributes with historical data|
|------|------|
|urn:oc:entityType:repeater:light | illuminance|
|urn:oc:entityType:iotdevice:vehicleSpeed | speed:median|
|urn:oc:entityType:iotdevice:magneticLoop | trafficIntensity, roadOccupancy, roadLoad|
|urn:oc:entityType:bikeStop | freeBikes, feeSpaces|
|urn:oc:entityType:iotdevice:irrigation | temperature:ambient, relativeHumidity, temperature:ground, tension:soilMoisture|
|urn:oc:entityType:mobileSensor | position:altitude, speed:instantaneous, direction:heading, mileage:total, chemicalAgentAtmosphericConcentration:CO, chemicalAgentAtmosphericConcentration:airParticles, temperature:ambient, relativeHumidity, chemicalAgentAtmosphericConcentration:O3|
|urn:oc:entityType:repeater:air | co_index, temperature:ambient|
|urn:oc:entityType:iotdevice:environmentalStation | temperature:ambient, humidity:relative, solarRadiation, rainFall, wind:speed, wind:direction, atmosphericPressure|
|urn:oc:entityType:repeater:temp | temperature:ambient|
|urn:oc:entityType:iotdevice:agriculture | temperature:ambient, relativeHumidity|
|urn:oc:entityType:repeater:noise | soundPressureLevel:ambient|
|urn:oc:entityType:iotdevice:parking | presenceStatus:parking|

### Examples

| Description | Query |
|------|------|
|Query assets in Santander of type urn:oc:entityType:iotdevice:magneticLoop. Page size is 10 and we query the 10th page  | https://discovery.organicity.eu/v0/assets/sites/santander?type=urn:oc:entityType:iotdevice:parking&per=10&page=10 |
|Query attributes with historical from asset urn:oc:entity:santander:traffic:magneticLoop:1004 | https://data.organicity.eu/urn:oc:entity:santander:parking:np3711/attributes |
|Query recorded values of attribute roadOccupancy from asset urn:oc:entity:santander:parking:np3711 for the Januar 2018 | https://data.organicity.eu/urn:oc:entity:santander:parking:np3711/presenceStatus:parking/readings?from=2018-01-01&to=2018-01-31 |



In [1]:
# import libraries
import urllib.request, urllib.parse, urllib.error
import ssl
import bs4 as bs
import pandas as pd
from datetime import datetime
import re
#set maximum lines to display when printing dataframe
pd.options.display.max_rows = 8
#To save data
import csv
from datetime import datetime

In [2]:
"""
Getting the entire available data set. The first step is to get Id and location for each device
"""

list_of_sensor_types = ["urn:oc:entityType:repeater:light", "urn:oc:entityType:iotdevice:vehicleSpeed", 
                      "urn:oc:entityType:iotdevice:magneticLoop", "urn:oc:entityType:bikeStop", 
                      "urn:oc:entityType:iotdevice:irrigation", "urn:oc:entityType:mobileSensor", 
                      "urn:oc:entityType:repeater:air", "urn:oc:entityType:iotdevice:environmentalStation", 
                      "urn:oc:entityType:repeater:temp", "urn:oc:entityType:iotdevice:agriculture", 
                      "urn:oc:entityType:repeater:noise", "urn:oc:entityType:iotdevice:parking"]

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

dfsensors = pd.DataFrame()
#The data we read from types
readings_types = ["id","latitude","longitude"]

#total number of sensors
total = 0

for sensor_type in list_of_sensor_types:
    url = "https://discovery.organicity.eu/v0/assets/sites/santander?type=" + sensor_type
    #print(url)
    connection = urllib.request.urlopen(url, context=ctx)
    sauce = urllib.request.urlopen(url, context=ctx)
    soup = bs.BeautifulSoup(sauce , 'html.parser')
    #print(soup)
    data_string = soup.get_text()
    #print(data_string)

    #Data cleaning
    #remove start and end of data_string
    data_string = data_string.replace("[{","")
    data_string = data_string.replace("]}","")
    
    #Counting sensors
    sensor_data = data_string.split("},{")
    print("There is ", len(sensor_data), " of", sensor_type)
    total = total + len(sensor_data)
    #remove site information from data
    sep = '\"site\"'
    sensor_data = [x.split(sep, 1)[0] for x in sensor_data]

    #Get, id, and position (latitude and longitude)
    for sensor in sensor_data:
        sensor = sensor.split("\"")
        sensor = list(filter(lambda a: a != ':', sensor))
        sensor_values = [sensor[sensor.index(x)+1] for x in readings_types]
        df_line = pd.DataFrame([sensor_values], columns=readings_types)
        dfsensors = dfsensors.append(df_line, ignore_index=True)

#Print the total number of sensors
print("The total number of sensors is ", total)
        
#Change coordinates from string to float
dfsensors.longitude = [float(x[1:len(x)-1]) for x in dfsensors.longitude]
dfsensors.latitude = [float(x[1:len(x)-1]) for x in dfsensors.latitude]
#Make sure that coordinates are ok, Note that check is limited to Western Europe
dfsensors["correct_longitude"] = [x if x < 0 else y for x,y in zip(dfsensors.longitude, dfsensors.latitude) ]
dfsensors["correct_latitude"] = [x if x > 0 else y for x,y in zip(dfsensors.longitude, dfsensors.latitude) ]
dfsensors = dfsensors.drop(columns=['longitude', 'latitude'])

dfsensors.columns = ["id","longitude","latitude"]

#Add line for ID number, to have just a number as identifier
dfsensors["id_number"] = [(x.split(":")[-2][0]).upper() + re.findall(r'\d+', x)[0] for x in dfsensors.id]

#Information to connect to page with data
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 

#Create dataframes for all sensor measurment available
df_all_sensor_data = pd.DataFrame()

for index, sensor_data in dfsensors.iterrows():
    url = "https://data.organicity.eu/" + sensor_data.id + "/attributes"
    #print(url)
    request= urllib.request.Request(url,None,headers) #The assembled request
    sauce = urllib.request.urlopen(request)
    soup = bs.BeautifulSoup(sauce , 'html.parser')
    data_string = soup.get_text()
    #print(data_string)
    number_of_attributes = 0
    #Data cleaning
    data_string = data_string.replace("{\"attributes\":[","")
    data_string = data_string.replace("]}","")
    #Get attributes
    attributes  = data_string.split(",")
    attributes = [x[1:-1] for x in attributes]
    
    #Create dataframes for actual sensor values, to data for every sensor seperately
    df_one_sensor = pd.DataFrame()

    if  attributes[0] == '':
        #print("No attributes at node " + str(sensor_data.id_number))
        continue 
    else:
        number_of_attributes = len(attributes)
        #print( "Attributes at node " + str(sensor_data.id_number) + ": " +(''.join(str(x) + ", " for x in attributes))[:-2])
        for attribute in attributes:
            #querry is the url
            url = "https://data.organicity.eu/" + sensor_data.id + "/" + attribute +"/readings"
            request= urllib.request.Request(url,None,headers) #The assembled request
            sauce = urllib.request.urlopen(request)
            soup = bs.BeautifulSoup(sauce , 'html.parser')
            data_string = soup.get_text()

            #Data cleaning
            #remove start and end of data because it is necesary
            data_string = data_string.replace("{\"readings\":[{","")
            data_string = data_string.replace("}]}","")
            #remove extra quotation marks
            data_string = data_string.replace("\"", "")
            data_string = data_string.replace("", "")
            data_string = data_string.replace("}]}","")
            measurments = data_string.split("},{")

            for each_measurment in measurments:
                s = each_measurment.split(",")
                measur_readings_types = []
                measur_values = []
                for test in s:
                    measur_readings_types.append(test.split(":",1)[0])
                    measur_readings_types.append("attribute")
                    measur_values.append(test.split(":",1)[1])
                    measur_values.append(attribute)
                #print(s[1].split(":"))    
                df_line = pd.DataFrame([measur_values], columns=measur_readings_types)
                #print(df_line)
                df_one_sensor = df_one_sensor.append(df_line, ignore_index=True)

        #Turn recvTime into a more usefull time tag, depending on application
        df_one_sensor.recvTime  = pd.to_datetime(df_one_sensor.recvTime, format="%Y-%m-%dT%H:%M:%S")
        date, time = zip(*[(d.date(), d.time()) for d in df_one_sensor['recvTime']])
        df_one_sensor = df_one_sensor.assign(date = date, time =time)
        df_one_sensor.index = pd.to_datetime(df_one_sensor['date'])
        df_one_sensor = df_one_sensor.drop('recvTime', 1)
        df_one_sensor = df_one_sensor.drop('date', 1)
        #print(df_one_sensor)
        #Save to a file
        df_one_sensor.to_csv("./scraped_data/" + sensor_data.id_number, sep='\t')
        #Save measurment to a dataframe saving all measurments
        df_all_sensor_data = df_all_sensor_data.append(df_one_sensor, ignore_index=True)
        
    #data_string = "illuminance"
    
    dfsensors["num_attributes"] = number_of_attributes
    dfsensors["attributes"] = (''.join(str(x) + ", " for x in attributes))[:-2]
#Save the two main files to a file
df_all_sensor_data.to_csv("./scraped_data/all_sensor_data", sep='\t')
dfsensors.to_csv("./scraped_data/all_sensor_metadata", sep='\t')

There is  30  of urn:oc:entityType:repeater:light
There is  21  of urn:oc:entityType:iotdevice:vehicleSpeed
There is  30  of urn:oc:entityType:iotdevice:magneticLoop
There is  16  of urn:oc:entityType:bikeStop
There is  12  of urn:oc:entityType:iotdevice:irrigation
There is  30  of urn:oc:entityType:mobileSensor
There is  1  of urn:oc:entityType:repeater:air
There is  3  of urn:oc:entityType:iotdevice:environmentalStation
There is  22  of urn:oc:entityType:repeater:temp
There is  9  of urn:oc:entityType:iotdevice:agriculture
There is  30  of urn:oc:entityType:repeater:noise
There is  30  of urn:oc:entityType:iotdevice:parking
The total number of sensors is  234


In [3]:
"""
A simple example to read data and save data to .csv of attribute roadOccupancy from asset 
urn:oc:entity:santander:parking:np3711 for the Januar 2018 
"""
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 

#querry is the url
url = "https://data.organicity.eu/urn:oc:entity:santander:parking:np3711/presenceStatus:parking/readings?from=2018-01-01&to=2018-01-31"

request= urllib.request.Request(url,None,headers) #The assembled request
sauce = urllib.request.urlopen(request)

soup = bs.BeautifulSoup(sauce , 'html.parser')

#print(soup)

data_string = soup.get_text()
#print(data_string)

#Data cleaning
#remove start and end of data because it is necesary
data_string = data_string.replace("{\"readings\":[{","")
data_string = data_string.replace("}]}","")
#remove extra quotation marks
data_string = data_string.replace("\"", "")
data_string = data_string.replace("", "")
data_string = data_string.replace("}]}","")
measurments = data_string.split("},{")

#print(measurments)


df = pd.DataFrame()

for each_measurment in measurments:
    s = each_measurment.split(",")
    measur_readings_types = []
    measur_values = []
    for test in s:
        measur_readings_types.append(test.split(":",1)[0])
        measur_values.append(test.split(":",1)[1])
    #print(s[1].split(":"))    
    df_line = pd.DataFrame([measur_values], columns=measur_readings_types)
    #print(df_line)
    df = df.append(df_line, ignore_index=True)
    
#Turn recvTime into a more usefull time tag, depending on application
df.recvTime  = pd.to_datetime(df.recvTime, format="%Y-%m-%dT%H:%M:%S")

date, time = zip(*[(d.date(), d.time()) for d in df['recvTime']])
df = df.assign(date = date, time =time)

#year, month, day, hour, minutes, seconds = zip(*[(d.year, d.month, d.day, d.hour, d.minute, d.second) for d in df['recvTime']])
#df = df.assign(year=year, month = month, day = day, hour=hour, minutes = minutes, seconds = seconds)

df.index = pd.to_datetime(df['date'])
df = df.drop('recvTime', 1)
df = df.drop('date', 1)
print(df)
#exaple, how to filter measurments based on the date
filtered = df['2018-01-01':'2018-01-02']
print(filtered)
#Save to a file
filtered.to_csv("./scraped_data/filtered_data", sep='\t')

           attrValue             time
date                                 
2018-01-01     false  14:43:58.480000
2018-01-01      true  14:58:59.019000
2018-01-01     false  23:22:59.623000
2018-01-02     false  11:08:01.175000
...              ...              ...
2018-01-22     false  09:41:53.118000
2018-01-22     false  13:44:52.499000
2018-01-24      true  10:12:56.475000
2018-01-24     false  10:29:57.080000

[106 rows x 2 columns]
           attrValue             time
date                                 
2018-01-01     false  14:43:58.480000
2018-01-01      true  14:58:59.019000
2018-01-01     false  23:22:59.623000
2018-01-02     false  11:08:01.175000
...              ...              ...
2018-01-02     false  14:53:12.123000
2018-01-02      true  15:15:02.229000
2018-01-02     false  16:02:01.226000
2018-01-02      true  19:45:01.757000

[13 rows x 2 columns]
